# Usage

## ~~Command line utility~~

~~TODO~~

```bash
$ # DOES NOT WORK YET r5py travel-time-matrix somewhere.osm.pbf somehwere_gtfs.zip --origins grid-centroids.gpkg ...
```

In [ ]:
# this cell is hidden from output
# it’s used to 
# - set sys.path to point to the local repo
# - make ipython show all outputs of each cell
# - provide a helper function to unobstrusively plot geometries

# - set sys.path to point to the local repo
import os.path
import sys
sys.path.append(
    os.path.join(
        os.path.abspath(""),  # directory of __file__,
        "../src/"
    )
)


# - make ipython show all outputs of each cell
get_ipython().ast_node_interactivity = "all"

## Python

R5py exposes some of R5’s functionality via its [Python API](reference.html), in a syntax similar to r5r’s. At the time of this writing, only the computation of travel time matrices has been fully implemented. 

Below, you find a minimal example of computing a travel time matrix:

### Load transport network

Virtually all operations of r5py require a transport network. As input files to construct a transport network topology, r5py needs the following data sets:

- The street network in the form of an OpenStreetMap extract covering the study area, prepared according to the instructions at https://docs.conveyal.com/prepare-inputs#preparing-the-osm-data, and possibly annotated with LADOT tags (see https://github.com/RSGInc/ladot_analysis_dataprep).
- Public transport schedules in GTFS format (optional, r5py can combine multiple GTFS files)

In this example, we use data from Helsinki metropolitan area, which you can find in the source code repository of r5py in `docs/data/`. The street network is a cropped and filtered extract of OpenStreetMap (© OpenStreetMap contributors, [ODbL license](https://www.openstreetmap.org/copyright)), the GTFS transport schedule is a cropped and minimised copy of Helsingin seudun liikenne’s (HSL) open data set [Creative Commons BY 4.0](https://www.hsl.fi/hsl/avoin-data#aineistojen-kayttoehdot).

To import the street and public transport networks, instantiate an `r5py.TransportNetwork` with the file paths to the OSM extract and the GTFS files:

In [ ]:
from r5py import TransportNetwork

transport_network = TransportNetwork(
    "data/kantakaupunki.osm.pbf",
    [
        "data/GTFS.zip"
    ]
)

### Compute a travel time matrix

A travel time matrix is a data set detailing the travel costs (e.g., time) from each point to each point in a set of origins and destinations in a study area. 

For the sake of this exercise, we will create a regular grid of points covering parts of Helsinki.

In [ ]:
import itertools
import geopandas
import numpy

south_west = (24.91, 60.19)
north_east = (24.97, 60.24)

coordinates =  itertools.product(
    numpy.arange(south_west[0], north_east[0], 0.005),
    numpy.arange(south_west[1], north_east[1], 0.005)
)

x, y = zip(*coordinates)

grid_points = geopandas.GeoDataFrame({
    "id": range(len(list(x))),
    "geometry": geopandas.GeoSeries.from_xy(
     x, y, crs="EPSG:4326"
    ),
    
})

grid_points

In [ ]:
grid_points.explore(marker_kwds={"radius": 6, "color": "red"})

Now, to compute a travel time matrix between all `grid_points`, we first instantiate an `r5py.TravelTimeMatrix` with the transport network we created earlier, and a list of origins (since we don’t specify a separate set of destinations, r5py will use the same points as destinations and produce a full set of origins and destinations). The constructor also accepts all parameters of [RegionalTask](reference.html#r5py.RegionalTask), such as transport modes, or walking speed. 

Calling `compute_travel_times()` on the instance will return a `pandas.DataFrame` with travel times between all points.

In [ ]:
import datetime

from r5py import TravelTimeMatrix, TransitMode, LegMode


travel_time_matrix = TravelTimeMatrix(
    transport_network,
    origins=grid_points,
    departure=datetime.datetime(2022,2,22,8,30),
    transport_modes=[TransitMode.TRANSIT, LegMode.WALK],
    #percentiles=[1,25,50,75,99],
    #breakdown=True
)
travel_times = travel_time_matrix.compute_travel_times()
travel_times

The values in the `travel_time` column are travel times between the points identified by `from_id` and `to_id`, in minutes. 

The median travel time to or from a certain point gives a good estimate of the overall accessibility to or from it, in relation to other points:

In [ ]:
travel_times.groupby("from_id")["travel_time"].median()

In [ ]:
travel_times.groupby("from_id")["travel_time"].median().median()